## Generatory
Generatory umożliwiają leniwe tworzenie sekwencji obiektów, które można iterować. Szczególnie przydatne są gdy wygenerowanie pojedynczego elementu sekwencji jest drogie - wymaga długich obliczeń, pobrania czegoś z sieci, zaalokowania dużej ilości pamieci, czytania z dysku etc. Dzięku generatorom nie jest konieczne generowanie wszystkich elementów na raz, co mogłoby być bardzo powolne lub zwyczajnie niemożliwe. Zamiast tego generatory mogą generować po jednym elemencie na raz, umożliwiając ich przetwarzanie na bierząco i umożliwiając niewygenerowanie elementów, które np. w przypadku szybszego zakończenia pętli nie byłby potrzebne. W swojej konstrukcji generatory to po postu funkcje, jednak zamiast słowa kluczowego `return` używa się `yield` - potencjalnie wielokrotnie, gdyż kontrola powraca w miejsce `yield` gdy pętla prosi o kolejny element (czyli woła na zwróconym iteratorze specjalną metodę `__next__()` - to jednak wykracza poza zakres tego szkolenia):

In [ ]:
class MyIter:
    def __iter__(self):
        ...
    def __next__(self):
        ...

In [ ]:
def even_number_generator(n):
    for i in range(n):
        print(f"generated {i}")
        yield i



for k in even_number_generator(10):
    print(f"got {k} from generator")

Można również tworzyć generatory przy użyciu comprehensions:

In [ ]:
def print_and_return(i):
    print(f"generated {i}")
    return i

for k in (print_and_return(i) for i in range(10)):
    print(f"got {k} from generator")

Generatory można też łączyć w łancuchy przy użyciu `yield from`:

In [ ]:
import time

def download_small_images():
    for i in range(10):
        # tu powinno być np. pobieranie obrazków z innego serwera
        time.sleep(1)
        yield f"mały_obrazek_{i}"

def download_medium_images():
    for i in range(10):
        # tu powinno być np. pobieranie obrazków z innego serwera
        time.sleep(1)
        yield f"średni_obrazek_{i}"


def download_big_images():
    for i in range(10):
        # tu powinno być np. pobieranie obrazków z innego serwera
        time.sleep(1)
        yield f"duży_obrazek_{i}"

def download_all_images_sequentially():
    yield "somethting"
    yield from download_small_images()
    yield from download_medium_images()
    yield from download_big_images()

def download_all_images_one_by_one():
    for small, medium, large in zip(download_small_images(), download_medium_images(), download_big_images()):
        yield small
        yield medium
        yield large

#for img in download_all_images_sequentially():
#    print(img)

for img in download_all_images_one_by_one():
    print(img)

Generatory mogą nie tylko produkować wartości, ale też je konsumować:

In [2]:
def hello():
    print("hello 1")
    counter = 0
    while True:
        print(f"counter: {counter}")
        value = yield counter
        print(f"po yield {value}")
        counter += 1

printer = hello()

print(next(printer))
print(next(printer))
print(next(printer))
print(printer.send("first message"))
printer.close()
# printer.send("second message")

# for i in range(10):
#     printer.send(f"looping - current index is {i}")

hello 1
counter: 0
0
po yield None
counter: 1
1
po yield None
counter: 2
2
po yield first message
counter: 3
3


Praktycznym i niezwiązanym z wydajnością zastosowaniem generatorów jest pisanie funkcji, które "otaczają" inne - słowo kluczowe `yield` wstrzymuje wykonanie funkcji "otaczającej" i oddaje kontrolę do funkcji wywołującej generator aż do ponownego wywołania lub powrotu z funkcji wywołującej. Wówczas wykonanie funkcji okalającej kontynuowane jest od linijki po słowie kluczowym `yield`. Przykładem może być pisanie managerów kontekstu z użyciem dekoratora `@contextlib.contextmanager`:

In [ ]:
from contextlib import contextmanager

@contextmanager
def managed_resource(*args, **kwds):
    # Code to acquire resource, e.g.:
    resource = acquire_resource(*args, **kwds)
    try:
        yield resource
    finally:
        # Code to release resource, e.g.:
        release_resource(resource)

with managed_resource() as r:
    zrob_cos_z_r(r)
    zrob_cos_jeszcze()

W powyższym przykładzie zamiast tworzyć klasę implementującą protokół contextmanagera (metody `__enter__` i `__exit__`) wystarczy napisać funkcję, która dokona alokacji i inicjalizacji zasobu przed `yield`, a zwolnienie i cleanup po. Warto zauważyć "opakowanie" linijki z `yield` w blok `try ... finally` - przekazujemy kontrolę do kodu poza naszym managerem i nie mamy pewności, że nie zostanie podniesiony wyjątek. `finally` w tym miejscu zapewnia, że zasób zostanie posprzątany. Funkcja musi też zwracać dokładnie jedną wartość - najlepiej zarządzany zasób, który zostanie przypisany do zmiennej w ramach klazuli `with ... as <zmienna>`. Kod generatora po `yield` zostanie wywołany, gdy blok kodu wewnątrz klauzuli `with ...` zostatnie w całości wykonany, tak samo jak funkcja `__exit__` w przypadku standardowej implementacji.


Analogiczny pomysł wykorzystuje `pytest` w mechanizmie `fixtures`:

In [ ]:
@pytest.fixture()
def tmp_dir_without_files(tmpdir):
    tmp = Path(tmpdir)
    tmp.mkdir(exist_ok=True)
    (tmp / "inner_folder").mkdir()
    yield tmp
    shutil.rmtree(tmp)